# Entrenar un modelo de clasificación con AutoML

In [ ]:
# mostrar info de azure-ai-ml

! pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


## Preparar datos

Usar losa datos de diabetes.

Recuperar el activo de datos especificando su nombre `diabetes-training-table` y versión `1`. 

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# Crea un conjunto de datos basado en los archivos en la carpeta de datos local
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configurar el job AutoML

In [ ]:
from azure.ai.ml import automl

# Configurar el job de clasificación
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# Establecer los límites
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# Establecer las propiedades de entrenamiento
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Ejecutar el job AutoML

In [ ]:
# Enviar el job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# URL
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)